In [1]:
data_dir = 'C:/Users/haile/Documents/GitHub/measles/data-ingest/inputs-measles/'
data_file = 'WHO Measles Caseload - July.xlsx'
metric_sheet = 'WEB'

metric_id = 6
data_source = 'WHO Measles Surveillance Data'

# database to use... metric for prod, metric_test for test
db_name = 'metric'
#db_name = 'metric_test'

In [ ]:
# put a list of keys we want to exclude from the metric upload, based on values 
# in the observation table (so we need place ids and dt_ids if using those)
# e.g.
# to exclude everything for Venezuela
# {'place_id': 244}
# 
# to exclude these two periods for Samoa
# {'place_id': 251, 'datetime_id': [14550, 14519]}
# 
# to exclude everything after Jan 2020 (since we don't have data yet)
# you can add a single value or a list of values to any key
# {'datetime_id': 14580}

exclusion_list = [
    {'place_id': 244},
    {'place_id': 251, 'datetime_id': [14550, 14519, 14580, 14611]},
    {'place_id': 48, 'datetime_id': [14277, 14305, 14336, 14366, 14397, 14427, 14458, 14489, 14519, 14550, 14580, 14611, 14642, 14671, 14702, 14732, 14763, 14793]},
    {'datetime_id': [14824, 14855, 14885, 14916, 14946]}
]

In [ ]:
import pandas as pd
import datetime
import json

In [4]:
from psycopg2.sql import SQL, Identifier, Placeholder
from psycopg2.extras import execute_batch

import psycopg2

from sqlalchemy import create_engine

In [5]:
from modules.connect import get_secret

In [6]:
def exclude(df, exclusion_list):
    for exclusion in exclusion_list:
        
        mask_list = []

        for key, value in exclusion.items():
            if type(value) is list:
                mask = (df[key].isin(value))
            else:
                mask = (df[key] == value)
            
            mask_list.append(mask)
        
        final_mask = mask_list[0]

        # if there are multiple conditions (e.g. time and place)
        # then the rows we want to exclude are only those that meet
        # all the conditions, so run through the masks and only
        # 'keep' the rows where the condition is true all the time
        # ('keep' because we'll invert the mask and actually exclude them)
        if len(mask_list) > 1:
            for mask in mask_list[1:]:
                final_mask = final_mask & mask
        
        old_len = df.shape[0]
        df = df[~final_mask]
        
        print(f"Went from {old_len} to {df.shape[0]} rows, excluded {old_len - df.shape[0]} rows")
        
    return(df)

In [7]:
rds_secret = json.loads(get_secret())

{'ARN': 'arn:aws:secretsmanager:us-west-1:560812044035:secret:talus_dev_rds_secret-okfjje', 'Name': 'talus_dev_rds_secret', 'VersionId': 'a04e03e6-cba7-467d-9a37-e2918e4d897f', 'SecretString': '{"username":"talus","password":"UKwwGaH5X8q688K4","engine":"postgres","host":"talus-dev.cvsrrvlopzxr.us-west-1.rds.amazonaws.com","port":5432,"dbname":"vaccines","dbInstanceIdentifier":"talus-dev","jwtSecretKey":"Wh;~e49[[BERw]bY"}', 'VersionStages': ['AWSCURRENT'], 'CreatedDate': datetime.datetime(2020, 3, 16, 15, 10, 47, 534000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'da066858-3969-4c53-8cb3-fb7e708b4290', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 17 Aug 2020 23:51:42 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '512', 'connection': 'keep-alive', 'x-amzn-requestid': 'da066858-3969-4c53-8cb3-fb7e708b4290'}, 'RetryAttempts': 0}}


In [8]:
def get_table(schema, table, engine):
    q_str = f"SELECT * FROM {schema}.{table}"
    return pd.read_sql(q_str, engine)

In [9]:
conn_str = "postgresql://{3}:{4}@{0}:{1}/{2}".format(rds_secret['host'],
                                                     '5432',
                                                     db_name,
                                                     rds_secret['username'],
                                                     rds_secret['password'])

engine = create_engine(conn_str)

In [10]:
dt_df = get_table('public', 'datetime', engine)

# dt_df = pd.DataFrame(dt_q, columns=['dt_id', 'day boolean', 'week_sunday', 'week_monday', 'month', 'year', 'dt'])

dt_df = dt_df.loc[(dt_df.month), :]

dt_df.loc[:, 'date'] = dt_df.dt.dt.date

#dt_df.head()

In [11]:
place_q = get_table('public', 'place', engine)

place_df = pd.DataFrame(place_q, columns=['place_id', 'name', 'description', 'fips', 'iso',
                                           'place_type', 'geom_type', 'poly_id', 'point_id',
                                           'iso2', 'region_sdg', 'region'])

#place_df.head()

In [12]:
data = pd.read_excel(data_dir + data_file, sheet_name=metric_sheet)

#data.head()

In [13]:
data_place = data.set_index('ISO3').join(place_df.loc[:,['place_id', 'iso']].set_index('iso')).reset_index().rename(columns={'index': 'iso'})

#data_place.head()

In [14]:
data_place.head()

,iso,Region,Country,Year,January,February,March,April,May,June,July,August,September,October,November,December,place_id
0,AFG,EMR,Afghanistan,2011,138.0,145.0,211.0,229.0,157.0,144.0,102.0,45.0,24.0,25.0,53.0,111.0,2
1,AFG,EMR,Afghanistan,2012,360.0,414.0,364.0,341.0,340.0,292.0,234.0,72.0,45.0,93.0,53.0,63.0,2
2,AFG,EMR,Afghanistan,2013,36.0,51.0,47.0,69.0,59.0,58.0,51.0,0.0,8.0,3.0,13.0,35.0,2
3,AFG,EMR,Afghanistan,2014,69.0,70.0,46.0,29.0,79.0,43.0,47.0,11.0,39.0,27.0,32.0,60.0,2
4,AFG,EMR,Afghanistan,2015,89.0,160.0,269.0,198.0,208.0,116.0,65.0,28.0,16.0,7.0,8.0,21.0,2


In [15]:
#data_place.loc[(data_place.iso == 'VEN'), :]

In [16]:
date_cols = ['place_id', 'Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

monthly = data_place.loc[:, date_cols].set_index(['place_id', 'Year']).stack().reset_index()

monthly.columns = ['place_id', 'year', 'month', 'value']

#monthly.head()

In [17]:
monthly.loc[:, 'date'] = monthly.apply(lambda x: datetime.datetime.strptime(f"{x['month']} 1, {x['year']}", "%B 1, %Y"), axis=1)

#monthly.head()

In [18]:
dated = monthly.set_index('date').join(dt_df.loc[:,['dt_id', 'date']].set_index('date')).reset_index(drop=True)

In [19]:
dated.loc[:, 'updated_at'] = datetime.datetime(2020, 8, 17) #.date()

dated.loc[:, 'metric_id'] = metric_id
dated.loc[:, 'data_source'] = data_source

dated = dated.loc[:, ['metric_id', 'value', 'updated_at', 'data_source', 'place_id', 'dt_id']].rename(columns= {'dt_id': 'datetime_id'})

dated.loc[:, 'updated_at'] = dated['updated_at'].dt.strftime('%Y-%m-%d')

In [20]:
dated.head()

,metric_id,value,updated_at,data_source,place_id,datetime_id
0,6,138.0,2020-08-17,WHO Measles Surveillance Data,2,11324
1,6,17.0,2020-08-17,WHO Measles Surveillance Data,3,11324
2,6,0.0,2020-08-17,WHO Measles Surveillance Data,5,11324
3,6,0.0,2020-08-17,WHO Measles Surveillance Data,7,11324
4,6,0.0,2020-08-17,WHO Measles Surveillance Data,8,11324


In [21]:
pre_excluded = dated.shape[0]
dated_excluded = exclude(dated, exclusion_list)
print(f"Went from {pre_excluded} to {dated_excluded.shape[0]} rows, excluded {pre_excluded - dated_excluded.shape[0]} rows")

Went from 20656 to 20540 rows, excluded 116 rows
Went from 20540 to 20537 rows, excluded 3 rows
Went from 20537 to 20519 rows, excluded 18 rows
Went from 20519 to 20347 rows, excluded 172 rows
Went from 20656 to 20347 rows, excluded 309 rows


In [22]:
row_list = tuple(tuple(row) for row in dated_excluded.to_numpy())

In [23]:
row_list[0:5]

((6, 138.0, '2020-08-17', 'WHO Measles Surveillance Data', 2, 11324),
 (6, 17.0, '2020-08-17', 'WHO Measles Surveillance Data', 3, 11324),
 (6, 0.0, '2020-08-17', 'WHO Measles Surveillance Data', 5, 11324),
 (6, 0.0, '2020-08-17', 'WHO Measles Surveillance Data', 7, 11324),
 (6, 0.0, '2020-08-17', 'WHO Measles Surveillance Data', 8, 11324))

In [24]:
len(row_list)

20347

In [25]:
test_q = """
INSERT INTO observation (
    metric_id,
    value,
    updated_at,
    data_source,
    place_id,
    datetime_id
)
VALUES (
    %s,
    %s,
    %s,
    %s,
    %s,
    %s
)
ON CONFLICT 
ON CONSTRAINT observation_metric_id_place_id_datetime_id_key
DO UPDATE SET (value, updated_at) = (EXCLUDED.value, EXCLUDED.updated_at);
"""

In [26]:
db_name

'metric'

In [27]:
rds_secret['host']

'talus-dev.cvsrrvlopzxr.us-west-1.rds.amazonaws.com'

In [28]:
# Open a cursor to perform database operations
connection_string = f"host={rds_secret['host']} dbname={db_name} user={rds_secret['username']} password={rds_secret['password']}"

conn = psycopg2.connect(connection_string)
cur = conn.cursor()

In [29]:
execute_batch(cur, test_q, row_list)

In [30]:
conn.commit()